In [1]:
import torch.optim.lr_scheduler as schedulers
from utils import retrieve_setup, retrieve_training_params, training_loop

/home/admin/miniconda3/envs/oml/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
model_name = "MobileNetV3Small" 
dataset_name = "CIFAR10"
lr_max = 0.0002
lr_min = 0.000001
n_epochs = 10

In [12]:
# retrieves
model, dataset = retrieve_setup(model_name, dataset_name)
optimizer, _, loss_fn, _, batch_size = retrieve_training_params(model, dataset_name, "parameters.yml")

# change lr from default to lr_max
for g in optimizer.param_groups:
    g["lr"] = lr_max

# implement linear increasing scheduler
total_iters = int(len(dataset["train"])/batch_size*n_epochs)
start_factor = lr_min/lr_max

scheduler = schedulers.LinearLR(
    optimizer, 
    start_factor=start_factor,
    end_factor=1,
    total_iters=total_iters
)
scheduler = ("rangeLR", scheduler)

Files already downloaded and verified
Files already downloaded and verified


In [13]:
# run training loop
_, losses = training_loop(
    model=model,
    dataset=dataset,
    scheduler=scheduler,
    optimizer=optimizer,
    loss_fn=loss_fn,
    n_epochs=n_epochs,
    batch_size=batch_size,
    test_strategy=("iter", 500),
    scheduler_strategy="iter",
    tensorboard_path=f"./tensorboard/{dataset_name}_{scheduler[0]}",
    device="cuda",
)